In [1]:
from ase import Atoms
import ase.units as units
from ase.io import read
from ase.build import bulk
from ase.calculators.espresso import Espresso

import numpy as np
import matplotlib.pyplot as plt

import os
import dotenv

dotenv.load_dotenv()
PWSCF_COMMAND = os.environ.get("PWSCF_COMMAND")
PPDIR = os.environ.get("PSEUDOPOTENTIALS")

outdir = "./results"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [2]:
# Define system and electron parameters
system_params = {
    'smearing': 'mp',
    'occupations': 'smearing',
    'starting_magnetization(1)': 0.7,
    'degauss': 0.02,
    'nspin': 2,
    'ntyp': 1,
    'ibrav': 0,
    'ecutwfc': 30 * units.eV,  # PW cutoff
    'ecutrho': 240 * units.eV,  # Charge cutoff
}


# Merge parameters into the input_data dictionary
input_data = {
    'system': system_params.copy(),
    'electrons': electron_params.copy()
}

In [6]:
iron_bcc = bulk('Fe', 'bcc', a=3.5)
print (iron_bcc.positions)

[[0. 0. 0.]]


In [3]:


k_points = 3

# Setup DFT calculation parameters
pseudopotentials = {'Fe': 'Fe.pbe-nd-rrkjus.UPF'}  # Example, change as needed

calc_params = {
    'calculation': 'scf',
    'tprnfor': True,
    'tstress': True,
    'input_data': {
        'system': {
            'ecutwfc': 30,  # PW cutoff
            'ecutrho': 300,  # Charge cutoff
        },
        'electrons': {
            'diagonalization': 'david',
            'mixing_beta': 0.5,
            'conv_thr': 1e-07,
        }
    },
    'pseudopotentials': pseudopotentials,
    'kpts': (k_points, k_points, k_points),  # k-points grid, adjust as needed
    'parallel': 'all',
    'directory': outdir,  # Custom directory for calculation files
    'label': "Fe",  # Prefix for the filenames
    'logfile': "Fe.log",  # Logfile name
    'command': "mpirun -np 16 " + PWSCF_COMMAND + " -in Fe.pwi > Fe.pwo",
    'pseudo_dir': PPDIR,
}

In [4]:
energies_bcc = []
energies_hcp = []

for lattice in ['bcc', 'hcp']:
    if lattice == 'bcc':
        for a_bcc in np.linspace(2.5, 5, 5):
            iron_bcc = bulk('Fe', 'bcc', a=a_bcc)

            # Attach the calculator to the structures
            calculator = Espresso(**calc_params)
            iron_bcc.set_calculator(calculator)
            energy_bcc = iron_bcc.get_potential_energy()
            energies_bcc.append(energy_bcc)


    elif lattice == 'hcp':
        for a_hcp, c_hcp in zip(np.linspace(2.5, 5, 5), np.linspace(3.5, 6, 5)):
            iron_hcp = bulk('Fe', 'hcp', a=a_hcp, c=c_hcp)

            # Attach the calculator to the structures
            calculator = Espresso(**calc_params)
            iron_hcp.set_calculator(calculator)
            energy_hcp = iron_hcp.get_potential_energy()
            energies_hcp.append(energy_hcp)


CalculationFailed: Calculator "espresso" failed with command "mpirun -np 16 mpirun -np 8 /opt/local/bin/pw.x -in Fe.pwi > Fe.pwo" failed in /Users/muon/Documents/2023-computational/alchemist/examples/fe/results with error code -15